# Purpose

This is a collection of functions specific to AWS S3 using boto3

# Discussion

this assumes that
* aws credentials have been previously setup
* Boto3 has been installed:  `pip3 install boto3`
* The following has been loaded globally:
  ``` python
import boto3
s3 = boto3.resource('s3')
```

# Functions

## create_bucket(bucket_name, s3_resource)

`bucket_create_response = create_bucket(bucket_name, s3_resource)`

In [ ]:
def create_bucket(bucket_name, s3_resource):
    return s3_resource.create_bucket(Bucket=bucket_name)

## searching for files

`get_matching_s3_objects(bucket, prefix="", suffix="")`

`key = get_matching_s3_keys(bucket, prefix="", suffix="")`

In [ ]:
#https://alexwlchan.net/2017/07/listing-s3-keys/
#https://alexwlchan.net/2019/07/listing-s3-keys/
def get_matching_s3_objects(bucket, prefix="", suffix=""):
    """
    Generate objects in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch objects whose key starts with
        this prefix (optional).
    :param suffix: Only fetch objects whose keys end with
        this suffix (optional).
    """
    s3 = boto3.client("s3")
    paginator = s3.get_paginator("list_objects_v2")

    kwargs = {'Bucket': bucket}

    # We can pass the prefix directly to the S3 API.  If the user has passed
    # a tuple or list of prefixes, we go through them one by one.
    if isinstance(prefix, str):
        prefixes = (prefix, )
    else:
        prefixes = prefix

    for key_prefix in prefixes:
        kwargs["Prefix"] = key_prefix

        for page in paginator.paginate(**kwargs):
            try:
                contents = page["Contents"]
            except KeyError:
                return

            for obj in contents:
                key = obj["Key"]
                if key.endswith(suffix):
                    yield obj


def get_matching_s3_keys(bucket, prefix="", suffix=""):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    :param suffix: Only fetch keys that end with this suffix (optional).
    """
    for obj in get_matching_s3_objects(bucket, prefix, suffix):
        yield obj["Key"]

#for key in get_matching_s3_keys(bucket='testname.asyla.org', prefix='BlueMarble/', suffix='.jpg'):
#    print(key)
#print('\n\n')
#for key in get_matching_s3_keys(bucket='testname.asyla.org', suffix=('.jpg', '.JPG')):
#    print(key)

## does_key_exist(bucket_name, file_name)

`boolean = does_key_exist(bucket_name, file_name)`

In [ ]:
#https://stackoverflow.com/questions/33842944/check-if-a-key-exists-in-a-bucket-in-s3-using-boto3

def does_key_exist(bucket_name, file_name):
    try:
        s3.Object(bucket_name, file_name).load()
    except:
        #print ('error')
        return (False)
    else:
        #print ('worked')
        return (True)

## updateDatabase(bucket, filename, recordId, recordData)

In [ ]:
#Update the database file
#
#This assumes the database
#  - resides in S3
#  - in YAML format
#  - it is a dictionary format
#  - and is small enough to reside in RAM

def updateDatabase(bucket, filename, recordId, recordData):
    from ruamel.yaml import YAML #[ruamel.yaml documentation](https://yaml.readthedocs.io/en/latest/index.html)
    yaml = YAML()
    
    dbContent = {} #create an empty dictionary
    
    #populate the datastructure with data if there is any data to work with
    if does_key_exist(bucket, filename):
        #fetch the DB and return it as a string
        #https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Object.get
        #https://stackoverflow.com/questions/31976273/open-s3-object-as-a-string-with-boto3#35376156
        fileBody = s3.Object(bucket, filename).get()['Body'].read().decode('utf-8') 
    
        dbContent = yaml.load(fileBody) #read the file into an OrderdDictionary
        #print (fileData)

    dbContent[recordId] = recordData #save the new record to the structure
    #print (dbContent)

    #https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Object.put
    result = s3.Object(bucket, filename).put( #save the data to a S3 key
        ACL='bucket-owner-full-control', #assume that this is not public info
        Body=yamlDump(dbContent), #convert the datastructure into YAML
        ContentEncoding='utf-8', #its a normal ascii file
        ContentType='text/plain' #declare its HTML so the browser will open it
    )
    
    return result

## readMarkDownFile

Read in a MarkDown file and return it as a string

`results = downloadTextFile (bucket, filename)`

* bucket: the S3 bucket
* filename: the name of the S3 key
* results = the file contents as a string

In [ ]:
def downloadTextFile (bucket, filename):
    return (s3.Object(bucket, filename).get()['Body'].read().decode('utf-8'))

## getKeyURL()

`results = getKeyURL(bucket, key)`

* results: url as a string
* bucket: the S3 bucket
* key: name of the S3 key

In [ ]:
#https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-presigned-urls.html
#note that this is doing more then just fetching the URL
#this is promising but is incomplete:  https://stackoverflow.com/a/48197877/12400492

def getKeyURL(bucket, key):
    url = boto3.client('s3').generate_presigned_url(
        'get_object',Params={
            'Bucket': bucket,
            'Key': key
        },
        ExpiresIn=60*60*24
    )
    
    #print (url)

    return (url.split('?')[0]) #for now, just remove the extra stuff